## Implementando o Mecanisco de Atencao em Python com uso de Framework

In [ ]:
# Destinado a instalacao de pacotes necessarios em notebooks Jupyter
%pip install -q -U watermark

In [3]:
%reload_ext watermark
%watermark -a "ibzoo"

Author: ibzoo



#### Instala e importa Pytorch

In [ ]:
%pip install -q -U torch
import torch
from torch import nn

#### A classe abaixo implmementa um modelo Transformer para aprendizado de sequencias usando o PyTorch

In [6]:
# Classe
class Transformer(nn.Module):

    # Metodo construtor
    def __init__(self, vocab_size, embedding_dim, n_heads, n_layers, dropout):

        # Inicializa o construtor da classe mae nn.Module
        super().__init__()
        
        # inicializa atributos dessa classe (self)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.dropout = dropout
        
        # Camada de Positional Encoding
        # Define a camada de embedding que transformara a sequencia de entrada em uma sequencia de vetores
        # de dimensao fixa (embedding_dim)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # NOVIDADE!!!!
        # Define o mecanismo de auto-atencao multi-headed
        # Sem esse mecanismo a arquitetura do Transformer se torna muito limitada.
        # Estabelece um criterio de modo que o modelo aprenda aquilo que e mais importante em cada sequencia
        # Dropout e usado para evitar overfitting(parametro p/ regularizacao). Seria o mesmo que o modelo aprender o detalhe de uma palavra depois da outra
        # precisamos que ele aprenda que depois de um artigo definido vem um substantivo, por exemplo.
        self.attention = nn.MultiheadAttention(embedding_dim, n_heads, dropout=dropout)
        
        # Define a rede neural feedforward que sera usada para gerar a sequencia de saida a partir da 
        # sequencia de entrada
        self.feed_forward = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )
        
        # Define a camada de saida final que transformara a sequencia de saida na forma de saida desejada
        self.out = nn.Linear(embedding_dim, vocab_size)

        # Metodo forward
        def forward(self, x):
            # Aplica a camada de embedding na sequencia de entrada
            x = self.embedding(x)
            
            # Aplica o mecanismo de auto-atencao multi-headed
            x = self.attention(x)
            
            # Aplica a rede neural feedforward
            x = self.feed_forward(x)
            
            # Aplica a camada de saida final
            x = self.out(x)
            
            return x

In [7]:
# Exemplo de uso
modelo = Transformer(vocab_size=1000, embedding_dim=32, n_heads=4, n_layers=2, dropout=0.5)

In [8]:
modelo.modules

<bound method Module.modules of Transformer(
  (embedding): Embedding(1000, 32)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
  )
  (feed_forward): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (out): Linear(in_features=32, out_features=1000, bias=True)
)>

In [9]:
modelo.attention

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
)